In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [20]:
#데이터 설명
with open('./data/crx.names') as t:
    print(t.read())

# A1 ~ A15 ::: feature
# A16 ::: target
# 결측치는 ?로 표기 되어 있음.


1. Title: Credit Approval

2. Sources: 
    (confidential)
    Submitted by quinlan@cs.su.oz.au

3.  Past Usage:

    See Quinlan,
    * "Simplifying decision trees", Int J Man-Machine Studies 27,
      Dec 1987, pp. 221-234.
    * "C4.5: Programs for Machine Learning", Morgan Kaufmann, Oct 1992
  
4.  Relevant Information:

    This file concerns credit card applications.  All attribute names
    and values have been changed to meaningless symbols to protect
    confidentiality of the data.
  
    This dataset is interesting because there is a good mix of
    attributes -- continuous, nominal with small numbers of
    values, and nominal with larger numbers of values.  There
    are also a few missing values.
  
5.  Number of Instances: 690

6.  Number of Attributes: 15 + class attribute

7.  Attribute Information:

    A1:	b, a.
    A2:	continuous.
    A3:	continuous.
    A4:	u, y, l, t.
    A5:	g, p, gg.
    A6:	c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
    A7:	v, h, bb, j, n, z

In [143]:
set_type = {'A1':str, 'A2':np.float64, 'A3':np.float64,'A4':str, 'A5':str,
           'A6':str, 'A7':str, 'A8':np.float64, 'A9':str, 'A10':str,
           'A11':np.float64, 'A12':str, 'A13':str, 'A14':np.float64, 'A15': np.float64, 'A16':str}

col_name = ['A'+str(c) for c in range(1,16+1)]
convert_value = lambda x: np.nan if x == '?' else x

In [144]:
df = pd.read_table('./data/crx.data', sep=',',header = None, names = col_name, converters= {c:convert_value for c in col_name})
df = df.astype(set_type)

In [145]:
df.dropna(inplace = True)

In [146]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1.0,f,g,202.0,0.0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43.0,560.0,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0.0,f,g,280.0,824.0,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100.0,3.0,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120.0,0.0,+


## Sci-kit Learn

In [198]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
model_dtr = DecisionTreeRegressor(random_state = 42)

In [199]:
df_x = df.drop(['A16'], axis = 1)
df_y = df[['A16']]

In [201]:
df_x = pd.merge(df_x.drop([c for c,b in (df_x.dtypes == object).reset_index().values if b == True], axis = 1),
         pd.get_dummies(df_x[[c for c,b in (df_x.dtypes == object).reset_index().values if b == True]]),
         how = 'left',
         left_index=True,
         right_index=True)

In [202]:
df_y['A16'] = df_y.A16.apply(lambda x: 0 if x == '+' else 1)

In [203]:
df = pd.merge(df_x, df_y, how = 'left', left_index = True, right_index = True)

In [204]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['A16'], axis = 1), df.A16, test_size=0.33, random_state=42)

In [206]:
model_dtr.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=42, splitter='best')

In [214]:
sum(model_dtr.predict(X_test) == y_test) / y_test.count()

0.8333333333333334